Data have been downloaded here : <a href="https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0209353">Plos One - World largest Study</a><br>
From data available only tables :
<ul>
    <li>S8 (baseline : metadata)</li>
    <li>S10 (blood cells parameters)</li>
    <li>S11 (blood parameters)</li>
    <li>S12 (weight, blood pressure, well being, ketones)</li>
    <li>S15 (lipid and glycemia parameters)</li>
</ul>
Were downloaded, a little pre-processing was done directly in Excel :
<ul>
    <li>keeping only the id and parameters, except for the metadata</li>
    <li>removing header and footer lines which were notes, from each file</li>
    <li>renaming the columns "lenght of fast (days)" by "length_of_fast"</li>
</ul>



In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Load the data
blood_cell           = pd.read_excel('blood_cell.xlsx')
blood_param          = pd.read_excel('blood_param.xlsx')
lipid_glucose        = pd.read_excel('lipid_glucose.xlsx')
metadata             = pd.read_excel('metadata.xlsx')
weight_sbp_wb_ketone = pd.read_excel('weight_sbp_wb_ketones.xlsx')

# Merge the dataframes on 'id' column
df_merge = (metadata
            .merge(weight_sbp_wb_ketone, on='id')
            .merge(lipid_glucose, on='id')
            .merge(blood_cell, on='id')
            .merge(blood_param, on='id'))


##### Cleaning : 
<ul>
    <li>transform the dataframe to a long format adding a column 'timepoint'</li>
    <li>cast columns to the right format replacing "<" by "" or "," by "."</li>
    <li>replace strong outliers by missing values</li>
</ul>

In [3]:
# delete the extra spaces in the column names
df_merge.columns = df_merge.columns.str.replace('  ', ' ')

# Get the columns ending with "pre" or "post"
columns_to_melt = df_merge.filter(regex='pre$|post$').columns
# Get metadata columns
id_vars_col = metadata.columns.tolist()

# Melt the dataframe
df_long = pd.melt(df_merge, 
                  id_vars=id_vars_col, 
                  value_vars=columns_to_melt, 
                  var_name='parameter')

## Extract the timepoint based on the suffix of the parameter column
df_long['timepoint'] = (df_long['parameter'].str
                        .endswith('post').astype(int))

## Extract the value name based on what comes before "pre" or "post"
df_long['parameter'] = (df_long['parameter'].str
                        .replace(r'\s*(pre|post)$', '', regex=True))

# Rename the value column in order to have like a categorical variable
df_long.timepoint = (df_long["timepoint"].astype(str)
                     .replace({"0":"pre","1":"post"}))

# Replace the "<" by "" in the value column to cast the columns as numeric
df_long.loc[:,"value"] = (df_long["value"].astype(str)
                          .replace({"<": "", ",":"."}, regex=True))
df_long["value"] = pd.to_numeric(df_long["value"], errors="coerce")

# replace strong outliers by np.nan, defined as values that are >= 5 std 
# from the mean of each group (parameter, timepoint)
df_long['value'] = (df_long
                    .groupby(['parameter', 'timepoint'])['value']
                    .transform(
    lambda x : x.mask(np.abs(((x - x.mean()) / x.std())).ge(5), np.nan)
    ))


##### Addition of informations in order to prepare the analysis

In [4]:
# addition of a numeric column indicating exactly the timepoint.
df_long["timeline"] = np.where(df_long["timepoint"] == "pre", 0, df_long["fasting duration (days)"])

df_long.sort_values(["parameter", "id", "timeline"], inplace=True)
df_long["value_change"] = (df_long
                           .groupby(["id", "parameter"])["value"]
                           .transform("diff"))
# we need to have the change in all rows of ('id', 'parameter') groups for the following steps
# using .bfill() or .ffill() introduce errors in the data, we prefer to use "first" in a transform function.
df_long["value_change"] = df_long.groupby(["id", "parameter"])["value_change"].transform("first")

# it still some outliers for the change, after having a look on it, 
# we decided to remove the values that are >= 8 std from the mean of each group (parameter, timepoint)
outlier_change_mask = (
    df_long
    .groupby(['parameter', 'timepoint'])['value_change']
    .transform(lambda x: np.abs((x - x.mean()) / x.std()).ge(7))
    )
df_long.loc[outlier_change_mask, 'value_change'] = np.nan
df_long.loc[outlier_change_mask, 'value'] = np.nan



In [5]:
df_wide = df_long.pivot_table(index=['id', 'timepoint', 'sex', 'fasting duration (days)', 'age (years)'], columns="parameter", values=["value", "value_change"], ).reset_index()
df_wide.columns = [" ".join(col) for col in df_wide.columns.values]
df_wide.columns = [col[6:] if col.startswith("value ") else col for col in df_wide.columns.values]
df_wide.columns = [col[13:] + " change" if col.startswith("value_change") else col for col in df_wide.columns.values]
df_wide.columns = [col.strip() for col in df_wide.columns.values]
df_wide["timepoint"] = df_wide["timepoint"].replace({"pre":0, "post":1})

# it will be usefill for having a jittered x axis on timepoint.
df_wide['jittered_x'] = (
    df_wide['timepoint']
    .apply(lambda x : x + round(np.random.uniform(-0.1, 0.1),2))
)
df_wide.sort_values(by=['timepoint','id'], inplace=True)


##### Export the cleaned dataframe to an Excel file

In [6]:
df_long.to_parquet('merged_data_long.parquet', index=False)
df_wide.to_parquet('merged_data_wide.parquet', index=False)

#### draw the plot of correlation exploration 

In [23]:
correlation_matrix = pd.read_excel("correlation_matrix.xlsx", index_col=0)


In [25]:

param = list_of_param[0]
filtered_matrix = correlation_matrix.loc[:,param].reset_index().set_index("index").fillna(0)
list_of_param =  filtered_matrix.index.to_list()
filtered_matrix.sort_values(by = param,inplace=True)
fig = go.Figure(data=go.Heatmap(
            z=filtered_matrix.values,
            y=filtered_matrix.index,
            x=filtered_matrix.columns,
            colorscale='RdBu',  # Change the colorscale to 'viridis'
            colorbar=dict(title='Correlation', orientation="v", thickness=15,tickfont=dict(size=7)),
            hoverongaps=False,
            text=[[name] for name in list(filtered_matrix.index)],
            texttemplate="%{text}",
            textfont={"size":8}
        ))
fig.update_layout(
        margin=dict(l=20, r=20, t=50, b=0),
        yaxis=dict(title="", showticklabels=False),
        xaxis=dict(title="", showticklabels=False),
        width=300,
        height=700
    )
fig.update_xaxes(title=f"Correlation : {param}")
fig


In [14]:
filtered_matrix

,AP (µkat/l) change
index,
baseline of the parameter,-0.436720
fasting duration (days),-0.180299
age (years),-0.020155
AP (µkat/l) change,1.000000
Acetoacetic acid (mg/dL) change,0.002779
BMI (kg/m²) change,0.069001
CRP (mg/l) change,0.142508
Ca (mmol/l) change,0.112219
Creatinine (µmol/l) change,0.071733
